## Expanding function to be used to expand theorems to help comprehension

In [1]:
import sys
sys.path.append("..")

from adapt_utils import replace_symbols

from tree_parser import *

import networkx as nx

#import matplotlib
#import matplotlib.pyplot as plt

import copy

import time

from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
from my_utils import get_proof_steps_graph, print_proof_steps_graph

import pickle

In [2]:
%%time

text = file_contents("../set.mm")
database = meta_math_database(text,n=3500)

included 5555695 tokens from ../set.mm
proposition: 3500CPU times: user 9.83 s, sys: 362 ms, total: 10.2 s
Wall time: 10.4 s


In [5]:
def eval_tree(tree):
    if tree.value in context_values: 
        return context_values[tree.value]
    
    prop = database.propositions[tree.value]
    assert len(tree.leaves)==len(prop.hyps)
    
    replacement_dict = {prop.hyps[i].variable: eval_tree(tree.leaves[i]) for i in range(len(prop.hyps)) if prop.hyps[i].type == "f"} #find the replacement rules for the variables
    #print 'replacement_dict', replacement_dict, 'applied to', prop.statement
    return string_replace(prop.statement[1:],replacement_dict)

def tree2str(tree):
    evaluated_tree = eval_tree(tree)
    return replace_symbols(" ".join(evaluated_tree))

def tree_to_string(tree, database, context):
    if tree.value in context.f: return context.f[tree.value].statement
    prop = database.propositions[tree.value]
    assert len(tree.leaves)==len(prop.hyps)
    replacement_dict = {prop.hyps[i].variable:tree_to_string(tree.leaves[i], database, context) for i in range(len(prop.hyps)) if prop.hyps[i].type == "f"} #find the replacement rules for the variables
    #print 'replacement_dict', replacement_dict, 'applied to', prop.statement
    return string_replace(prop.statement[1:],replacement_dict)

In [54]:
def tree2ptree(tree, ph_dict={}):

    new_leaves = []
    for l in tree.leaves:
        if str(l) not in ph_dict:
            ph_dict[str(l)] = tree2ptree(l, ph_dict)
        new_leaves.append(ph_dict[str(l)])
    
    return PTree(tree.value, new_leaves)

In [55]:
prop = database.propositions["bitr4i"]
tree2ptree(prop.tree).statement

'( 𝜑 ↔ 𝜒 )'

In [7]:
def get_prop_hyps(prop):
    #We should use this function instead of direct checking for hypothesis
    #because if the prop is already a hypothesis, this property will not exist
    #which will raise an exception. This encapsulates and resolves this
    return prop.hyps if hasattr(prop, "hyps") else []

def get_step_replace_dict(step):
    
    repdict = {}
    
    #In case we are dealing with already hypotehsis nodes, there no hypothesis to work with
    #since there is no hypotehsis of a hypothesis, so we may skip it here
    #if hasattr(step.prop, "hyps"):
    
    e_hyps = [h for h in get_prop_hyps(step.prop) if h.type == 'e']
    assert len(e_hyps) == len(step._prior_entails), print(len(e_hyps), len(step._prior_entails))

    #Populate with hypothesis
    for raw_hyp, rep_hyp in zip(e_hyps, step._prior_entails):

        raw_tree = raw_hyp.tree
        rep_tree = rep_hyp.tree

        for pos in raw_tree.breadth_first_position_list():
            raw_subtree = raw_tree.tree_at_position(pos)
            rep_subtree = rep_tree.tree_at_position(pos)
            if raw_subtree.value != rep_subtree.value:
                repdict[raw_subtree.value] = rep_subtree

    
    
    #Populate with conclusion (this is necessary when there is no hypothesis)
    raw_tree = step.prop.tree
    rep_tree = step.tree
             
    for pos in raw_tree.breadth_first_position_list():
        raw_subtree = raw_tree.tree_at_position(pos)
        rep_subtree = rep_tree.tree_at_position(pos)
        if raw_subtree.value != rep_subtree.value:
            if raw_subtree.value in repdict and rep_subtree != repdict[raw_subtree.value]:
                print(rep_subtree, repdict[raw_subtree.value])
                raise Exception("Different trees within same node!")
            else:
                repdict[raw_subtree.value] = rep_subtree
        
    return repdict

The problem with ax-mp may be solvable by using the previous idea of propagate context, although the previous appraoch failled with dfss3


In [17]:
def populate_post_entails(step):
    #Get all steps. This is necessary because hypothesis are not natively within entails proof_steps
    all_steps = []
    for pstep in step.entails_proof_steps:
        if pstep not in all_steps:
            all_steps.append(pstep)

        for prior_pstep in pstep._prior_entails:
            if prior_pstep not in all_steps:
                all_steps.append(prior_pstep)

    for child_step in all_steps:
        child_step._post_entail = None
        
    for child_step in all_steps:
        for gchild_step in child_step._prior_entails:
            gchild_step._post_entail = child_step

In [ ]:
#root_step = database.propositions["dfss3"].entails_proof_steps[0]

In [8]:
prop = database.propositions["dfss2"]

root_step = prop.entails_proof_steps[-1]

for exp_step in prop.entails_proof_steps:
    print("--- [{}]".format(exp_step.prop.label), len(exp_step._prior_entails), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

print()

exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

--- [dfss] 0 |- ( A ⊆ B ↔ A = ( A ∩ B ) )
--- [df-in] 0 |- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
--- [eqeq2i] 1 |- ( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
--- [abeq2] 0 |- ( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [3bitri] 3 |- ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [pm4.71] 0 |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [albii] 1 |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
--- [bitr4i] 2 |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )

[bitr4i] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
--- [bicomi] |- ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )
--- [bitri] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [9]:
root_step = exp_proof[0]

exp_proof = expand_proof_step(root_step)

print("[{}]".format(root_step.prop.label), "|-", replace_symbols(root_step.tree.eval(database, root_step.context)))
print()

for exp_step in exp_proof:
    print("--- [{}]".format(exp_step.prop.label), "|-", replace_symbols(exp_step.tree.eval(database, root_step.context)))

[bicomi] |- ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )

--- [bicom1] |- ( ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ) → ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) ) )
--- [ax-mp] |- ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [28]:
step_rep_dict = get_step_replace_dict(root_step)
step_rep_dict

{'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
 'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>}

### Defining Classes

In [5]:
def expand_proof_step(root_step):
    
    expanded_steps = []
    
    if not hasattr(root_step, "replace_dict_list"):
        root_step.replace_dict_list = [get_step_replace_dict(root_step)]
        
    step2exp = dict() #Store references from steps to expanded steps to populate 
    
    for child_step in root_step.prop.entails_proof_steps:
        #Take the original prop tree
        exp_tree = child_step.prop.tree.copy()
        
        #Take the step transformation that was used to the raw step to the replace step
        replace_dict_list = [get_step_replace_dict(child_step)]
            
        #Get the root step replace dicts so we can iterativelly expand the steps
        replace_dict_list.extend(root_step.replace_dict_list)
        
        for rep_dict in replace_dict_list:
            exp_tree = exp_tree.replace(rep_dict)
        
        prior_statements = child_step.prior_statements if hasattr(child_step, "prior_statements") else []
        
        exp_step = proof_step(exp_tree, root_step.context, child_step.prop, prior_statements)
        exp_step.replace_dict_list = replace_dict_list
        
        expanded_steps.append(exp_step)
        
        step2exp[child_step] = exp_step
        
    #Populate _prior_entails
    for child_step in root_step.prop.entails_proof_steps:
        step2exp[child_step]._prior_entails = [step2exp.get(cc, cc) for cc in child_step._prior_entails]
    
        
    return expanded_steps